<a href="https://colab.research.google.com/github/elianarodriggs/analilse_csat/blob/main/case_csat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
pip install nbconvert

In [ ]:
## LENDO BASES DE DADOS

tickets = pd.read_csv('/content/tb_tickets.csv')
csat = pd.read_csv('/content/tb_csat.csv')
grandes_contas = pd.read_csv('/content/tb_grandes_contas.csv')

In [ ]:
tickets['dt_hr_criacao'] # valor original da coluna de data criacao

,dt_hr_criacao
0,2023-10-03T12:46:26.7236075
1,2023-10-03T13:59:28.3953558
2,2023-10-03T14:40:27.5606103
3,2023-10-03T18:23:43.3335535
4,2023-10-04T13:20:01.7574209
...,...
43368,2024-01-03T12:14:50.1524568
43369,2024-01-19T14:04:37.5895408
43370,2023-10-09T14:39:35.3642785
43371,2023-10-28T12:04:40.7021481


In [ ]:
## convertendo valores para datetime

tickets['dt_hr_criacao'] = pd.to_datetime(tickets['dt_hr_criacao'], errors='coerce').dt.date
tickets['dt_hr_fechamento'] = pd.to_datetime(tickets['dt_hr_fechamento'], errors='coerce').dt.date

In [ ]:
tickets['dt_hr_criacao'] # resultado da conversao da coluna de data criacao

,dt_hr_criacao
0,2023-10-03
1,2023-10-03
2,2023-10-03
3,2023-10-03
4,2023-10-04
...,...
43368,2024-01-03
43369,2024-01-19
43370,2023-10-09
43371,2023-10-28


In [ ]:
## TRATAMENTO DE DADOS - BASE CSAT

# conversao para datetime
csat['dt_hr_resposta'] = pd.to_datetime(csat['dt_hr_resposta'], errors='coerce').dt.date

# nan para nulo
csat['des_comentario'] = csat['des_comentario'].fillna("")

## TRATAMENTO DE DADOS - BASE CSAT

In [ ]:
## TRATAMENTO DE DADOS - TICKETS

# converte a coluna 'cod_cliente' pra float
tickets['cod_cliente'] = tickets['cod_cliente'].astype(float)
# substitui NaN por 0
tickets['cod_cliente'] = tickets['cod_cliente'].fillna(0)
# converte de float para int
tickets['cod_cliente'] = tickets['cod_cliente'].astype(int)
# converte de int para str
tickets['cod_cliente'] = tickets['cod_cliente'].astype(str)

## TRATAMENTO DE DADOS - TICKETS

In [ ]:
## TRATAMENTO DE DADOS - GRANDES CONTAS

grandes_contas.rename(columns={'grandes_contas': 'cod_cliente'}, inplace=True)
grandes_contas['cod_cliente'] = grandes_contas['cod_cliente'].astype(str)

## TRATAMENTO DE DADOS - GRANDES CONTAS

In [ ]:
## CRUZANDO DADOS DA BASE CSAT E TICKETS

csat_tickets = csat.merge(tickets, on='cod_ticket', how='left')

In [ ]:
## CRUZANDO BASE (TICKES/CSAT) COM A BASE DE GRANDES CLIENTES

df = csat_tickets.merge(grandes_contas, on='cod_cliente', how='left')

In [ ]:

# CRIANDO FLAG PRA IDENTIFICAR GRANDES CLIENTES NO DATAFRAME
df['flag_grandes_contas'] = np.where(df['cod_cliente'].isin(grandes_contas['cod_cliente']), 1, 0)

## CALCULO INDICADORES

Volume de Tickets

In [ ]:
volume_tickets = tickets['cod_ticket'].nunique()
print(f"% O volume de tickets é: {volume_tickets}")

% O volume de tickets é: 43373


 % FCR (First Call Resolution)

In [ ]:
## soma do flag resolvido n1 divido pela quantidade de tickets

fcr = tickets['flg_resolvido_n1'].sum() / tickets['cod_ticket'].nunique() * 100
print(f"% FCR: {fcr:.2f}%")

% FCR: 18.76%


TMR (Tempo Médio de Resolução)

In [ ]:
def tempo_medio_resolucao(tickets):

 # convertendo para datetime
    tickets['dt_hr_fechamento'] = pd.to_datetime(tickets['dt_hr_fechamento'])
    tickets['dt_hr_criacao'] = pd.to_datetime(tickets['dt_hr_criacao'])

    # Calculando a diferença em dias corridos
    tickets['tempo_resolucao'] = (tickets['dt_hr_fechamento'] - tickets['dt_hr_criacao']).dt.days


    # calcular a média
    tmr = tickets['tempo_resolucao'].mean()

    return tmr

# Calcular tempo médio de resolução
tempo_medio = tempo_medio_resolucao(tickets)
print(f"Tempo Médio de Resolução (TMR): {tempo_medio:.2f} dias")

Tempo Médio de Resolução (TMR): 1.31 dias


% SLA (Cumprimento do SLA)

In [ ]:
# prazo sla
sla_1 = 1 # sla de 1 dia
sla_2 = 2 # sla de 2 dias
sla_3 = 3 # sla de 3 dias

tickets['sla'] = sla_3  # Como exemplo, assumindo que todos têm o SLA de 1 dia

# Tickets dentro do SLA de 24 horas
tickets_dentro_sla = (tickets['tempo_resolucao'] <= tickets['sla']).sum()

# Tickets fora do SLA de 24 horas
tickets_fora_sla = (tickets['tempo_resolucao'] > tickets['sla']).sum()

print(f"Quantidade de tickets dentro do SLA (<= {sla_3} dia): {tickets_dentro_sla}")
print(f"Quantidade de tickets fora do SLA (> {sla_3} dia): {tickets_fora_sla}")

# Calcular % SLA
sla_percentual = (tickets['tempo_resolucao'] <= tickets['sla']).sum() / tickets['cod_ticket'].nunique() * 100
print(f"% SLA: {sla_percentual:.2f}%")

Quantidade de tickets dentro do SLA (<= 3 dia): 39467
Quantidade de tickets fora do SLA (> 3 dia): 2977
% SLA: 90.99%


% Envio de CSAT

In [ ]:
total_tickets = tickets['cod_cliente'].nunique()

# Contar o total de pesquisas enviadas - base csat
total_enviadas = df['cod_cliente'].nunique()

# Calcular o % de envio
percentual_envio = (total_enviadas / total_tickets) * 100

print(f'total enviadas {total_enviadas}')
print(f'total tickets {total_tickets}')
print(f"% de Envio: {percentual_envio:.2f}%")

total enviadas 2017
total tickets 5834
% de Envio: 34.57%


% Resposta de CSAT

In [ ]:
total_tickets = tickets['cod_cliente'].nunique()

# Contar o total de pesquisas enviadas - base csat
total_enviadas = df['cod_cliente'].nunique()

# Calcular o % de envio
percentual_envio = (total_enviadas / total_tickets) * 100

print(f'total enviadas {total_enviadas}')
print(f'total tickets {total_tickets}')
print(f"% de Resposta: {percentual_envio:.2f}%")

total enviadas 2017
total tickets 5834
% de Resposta: 34.57%


% CSAT (Nota de Satisfação)

In [ ]:
def convert_to_numeric(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return np.nan


# Aplicar a função na coluna 'nota'
df['nota_numero'] = df['nota'].apply(convert_to_numeric)

## criei a coluna 'nota_numero' com a transformacao dela só pra int pros calculos abaixo
df['nota_numero'] = df['nota_numero'].fillna(0).astype(int)

In [ ]:
# notas consideradas satisfatorias
nota_satisfacao = [8,9,10]

# tickets com nota de satisfação 8, 9 e 10 na base csat
tickets_satisfatorios = df['nota_numero'].isin(nota_satisfacao).sum()

# tickets da base csat
total_tickets = tickets['cod_cliente'].nunique()

# calculo de % csat
csat_percentual = (tickets_satisfatorios / total_tickets) * 100

print(f'total tickets csat: {total_tickets}')
print(f'total tickets satisfatorios: {tickets_satisfatorios}')

print(f"% CSAT: {csat_percentual:.2f}%")

total tickets csat: 5834
total tickets satisfatorios: 5120
% CSAT: 87.76%


% Resolutividade de CSAT

In [ ]:

resolvido_sum = csat_tickets[df['nota'] == 'Resolvido'].shape[0]
perg_resolvido_sum = csat_tickets[df['des_pergunta'] == 'O seu problema foi resolvido?'].shape[0]

resolutividade_csat = resolvido_sum / perg_resolvido_sum * 100

print(f"Qtd Resolvidos: {resolvido_sum}")
print(f"Total CSAT: {perg_resolvido_sum}")
print(f"% Resolutividade de CSAT: {resolutividade_csat:.2f}%")

Qtd Resolvidos: 5024
Total CSAT: 6273
% Resolutividade de CSAT: 80.09%


## Analise de negocio

In [ ]:
# Qual a taxa de contatos por cliente?

taxa_contatos = tickets['cod_ticket'].nunique() / tickets['cod_cliente'].dropna().nunique()

print(f"Taxa de contatos por cliente: {taxa_contatos:.2f}")

Taxa de contatos por cliente: 7.43


In [ ]:
# Existem clientes “heavy users” do atendimento?

# Contagem de tickets por cliente
tickets_por_cliente = tickets['cod_cliente'].value_counts()

# Heavy users: clientes com mais de 3x a média de tickets
media_tickets = tickets_por_cliente.mean()
heavy_users = tickets_por_cliente[tickets_por_cliente > 3 * media_tickets]
print("Clientes heavy users:")
print(heavy_users)

Clientes heavy users:
cod_cliente
0        1166
3143      477
11310     323
4688      217
9394      208
         ... 
3578       23
422        23
3098       23
718        23
7179       23
Name: count, Length: 366, dtype: int64


In [ ]:
# Quais os clientes menos satisfeitos?

clientes_insatisfeitos = df.groupby('cod_cliente')['nota_numero'].mean().nsmallest(10)
print("Clientes menos satisfeitos:")
print(clientes_insatisfeitos)

Clientes menos satisfeitos:
cod_cliente
10003    0.0
10008    0.0
10164    0.0
10244    0.0
10297    0.0
10317    0.0
1038     0.0
10430    0.0
10479    0.0
10484    0.0
Name: nota_numero, dtype: float64


In [ ]:
# Existe diferença entre a satisfação do cliente “grandes contas” e o cliente “padrão”?

csat_grandes = df[df['flag_grandes_contas'] == 1]['nota_numero'].mean()
csat_padrao = df[df['flag_grandes_contas'] == 0]['nota_numero'].mean()

print(f"Nota média CSAT - Grandes Contas: {csat_grandes:.2f}")
print(f"Nota média CSAT - Clientes Padrão: {csat_padrao:.2f}")


Nota média CSAT - Grandes Contas: 4.09
Nota média CSAT - Clientes Padrão: 4.34


In [ ]:
# Quais são os produtos com maior taxa de contato?

# Extrair produto da árvore de atendimento
df['produto'] = df['desc_arvore_completa'].str.split(',').str[0]

produtos_contato = df['produto'].value_counts().head(10)
print("Produtos com maior taxa de contato:")
print(produtos_contato)


Produtos com maior taxa de contato:
produto
['🌳 Atendimento SysPDV'               6887
['🌳 Atendimento Varejofacil'          2217
['🌳 Movidesk'                         1620
['🌳 Atendimento TEF'                   667
['🌳 Atendimento Milênio'               458
['💻 Atendimento SysPDV Web'             62
['🌳 Atendimento SysPDV']                27
['💻  Atendimento Customer Success'      17
['💻 Atendimento Migração'               16
['🌳 Atendimento Varejofacil']            4
Name: count, dtype: int64


In [ ]:
#Quais são os temas mais críticos?

df['tema'] = df['desc_arvore_completa'].str.split(',').str[-1]

temas_criticos = df['tema'].value_counts().head(10)
print("Temas mais críticos:")
print(temas_criticos)

Temas mais críticos:
tema
'Troca de controle']                      1750
'Abandono CHAT']                          1546
'Gerar (Nova ou Expirada) ']               748
'Informações de Chamados ( Tickets )']     282
'Rejeições ( NF Saida )']                  211
'Configuração TEF / Modulos']              189
'Chave VPN']                               176
'Utilização NFe/NFCe (TecnoSpeed)']        165
'Produto']                                 134
'Informações Tarefas']                     130
Name: count, dtype: int64
